# Equality & Identity

- `is` checks if they are the same person
- `==` checks if they have the same face

Short explanation:

- An `is` expression evaluates to True if two variables point to the
same (identical) object.
- An `==` expression evaluates to True if the objects referred to by
the variables are equal (have the same contents).

## List

In [9]:
a = [1, 2, 3]
b = a

a is b, a == b

(True, True)

In [10]:
c = [1, 2, 3]

a is c, a == c

(False, True)

## Class

In [28]:
class Person:
    def __init__(self, name):
        self.name = name

    def __eq__(self, other):
        # by default __eq__ `==` is same as `is` operator
        return self.name == other.name

In [32]:
p1 = Person('jon')
p2 = Person('sam')
p3 = p1
p4 = Person('jon')

id(p1), id(p2), id(p3), id(p4)

(1506095636048, 1506095960912, 1506095636048, 1506095962432)

In [36]:
p1 is p2, p1 is p3, p1 is p4

(False, True, False)

In [38]:
p1 == p2, p1 == p3, p1 == p4

(False, True, True)

In [34]:
# p1 is an instance while Person is a class
# they occupy different memory locations
p1 is Person, isinstance(p1, Person)

(False, True)

# str & repr

In [11]:
class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage

    def __repr__(self):
        return '__repr__ for Car'

    def __str__(self):
        # calls repr by default
        return '__str__ for Car'


my_car = Car('red', 37281)

In [12]:
print(my_car)  # calls __str__

__str__ for Car


In [13]:
my_car  # calls __repr__

__repr__ for Car

In [15]:
class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage

    def __repr__(self):
        return f'{self.__class__.__name__}({self.color!r}, {self.mileage!r})'

    def __str__(self):
        return f'a {self.color} car'


my_car = Car('red', 37281)

In [16]:
print(my_car)  # calls __str__

a red car


In [17]:
my_car  # calls __repr__

Car('red', 37281)

# MRO

In [39]:
class E:
    def greet(self):
        print("E.greet")

class A:
    def greet(self):
        print("A.greet")
        # noinspection PyUnresolvedReferences
        super().greet()  # IDE is wrong

class B(A, E):
    def greet(self):
        print("B.greet")
        super().greet()

class C(A):
    def greet(self):
        print("C.greet")
        super().greet()

class D(B, C):
    def greet(self):
        print("D.greet")
        super().greet()


d = D()
d.greet()

D.greet
B.greet
C.greet
A.greet
E.greet


## Hierarchy

        E
        │
        A
      /   \
     B     C
       \ /
        D

In [40]:
D.mro()

[__main__.D, __main__.B, __main__.C, __main__.A, __main__.E, object]

## super()

`super()` calls the next method in MRO

# ABC

In [44]:
from abc import ABC, abstractmethod


class Animal(ABC):
    @abstractmethod
    def sound(self):
        pass

    @abstractmethod
    def greet(self):
        print("Base greeting")


class Dog(Animal):
    def sound(self):
        print("woof")

    def greet(self):
        super().greet()
        print("Y greeting")

In [45]:
d = Dog()
d.sound()
d.greet()

woof
Base greeting
Y greeting


# Virtual Subclass is deprecated

In [ ]:
from abc import ABC, abstractmethod

class Flyer(ABC):
    @abstractmethod
    def fly(self):
        pass

class Airplane:
    def fly(self):
        print("Zoom")

# Register Airplane as a *virtual* subclass of Flyer
# register() is defined in ABC
Flyer.register(Airplane)
# python will just believe you.
# it assumes that Airplane implements all abstract methods of Flyer.
# at runtime you may get errors if it doesn't.

print(issubclass(Airplane, Flyer))     # True
print(isinstance(Airplane(), Flyer))   # True

# Protocol

In [47]:
from typing import Protocol, runtime_checkable


@runtime_checkable
class Renderable(Protocol):
    def render(self) -> str:
        pass


class HTML:
    def render(self) -> str:
        return "<p>Hello</p>"


class PDF:
    def render(self) -> str:
        return "PDF bytes"


def show(x: Renderable):
    """
    Both HTML and PDF satisfy the protocol,
    even though they don’t inherit from it.

    By default, it is only enforced by static type checkers,
    not at runtime.
    """
    print(x.render())


show(HTML())
show(PDF())

<p>Hello</p>
PDF bytes


In [48]:
# Without runtime_checkable, that's a TypeError.
isinstance(HTML(), Renderable)

True

# Property

In [49]:
class Person:
    def __init__(self, age):
        self._age = age

    @property
    def age(self):
        return self._age

    @age.setter
    def age(self, value):
        if value < 0:
            raise ValueError("Negative age not allowed")
        self._age = value


p = Person(25)
print(p.age)
p.age = 30
print(p.age)

25
30


# Cached Property

In [50]:
from functools import cached_property

class Data:
    @cached_property
    def result(self):
        print("Computing...")
        return 42


data = Data()
print(data.result)  # Computes and caches the result
print(data.result)  # Returns cached result without recomputing

Computing...
42
42


# Monkey Patching

Monkey patching is dynamically changing a module, class, or function at runtime, to
add features or fix bugs.

In [7]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]


deck = FrenchDeck()
deck[:3]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

In [10]:
from random import shuffle
# TypeError: 'FrenchDeck' object does not support item assignment
# shuffle(deck)

In [9]:
def set_card(deck, position, card):
    deck._cards[position] = card


FrenchDeck.__setitem__ = set_card
shuffle(deck)
deck[:3]

[Card(rank='K', suit='diamonds'),
 Card(rank='J', suit='spades'),
 Card(rank='10', suit='spades')]

# Meta Programming

## `__init_subclass__`

In [4]:
class Parent:
    def __init_subclass__(cls, **kwargs):
        # runs automatically whenever a class is subclassed.
        # called when subclass is created, not when instance is created.
        print("Subclass created:", cls.__name__)


class Child(Parent):
    pass

"""
Good for plugin appends and enforcing rules on subclasses.
"""

Subclass created: Child


## Class Decorator

In [5]:
def add_repr(cls):
    def __repr__(self):
        return f"<{cls.__name__} {self.__dict__}>"
    cls.__repr__ = __repr__
    return cls


@add_repr
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y


Point(2, 3)

<Point {'x': 2, 'y': 3}>

## Dynamic Attributes

In [4]:
class Quantity:
    # use set_name instead to set storage_name
    # def __init__(self, storage_name):
    #     self.storage_name = storage_name

    def __set_name__(self, owner, name):
        self.storage_name = name

    def __set__(self, instance, value):
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            msg = f'{self.storage_name} must be > 0'
            raise ValueError(msg)

    def __get__(self, instance, owner):
        if instance is None:
            """
            When an attribute is accessed through a class,
                the descriptor receives instance=None.

            If instance is None,
                the descriptor should return itself.
            """
            return self
        else:
            return instance.__dict__[self.storage_name]


class LineItem:
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price


## Restrict Attributes with __slots__

In [1]:
class Point:
    __slots__ = ('x', 'y')

    def __init__(self, x, y):
        self.x = x
        self.y = y

## Class Factory

In [5]:
from typing import Union, Any
from collections.abc import Iterable, Iterator

FieldNames = Union[str, Iterable[str]]


def parse_identifiers(names: FieldNames) -> tuple[str, ...]:
    if isinstance(names, str):
        names = names.replace(',', ' ').split()

    if not all(s.isidentifier() for s in names):
        raise ValueError('names must all be valid identifiers')

    return tuple(names)


def record_factory(cls_name: str, field_names: FieldNames) -> type[tuple]:
    slots = parse_identifiers(field_names)

    def __init__(self, *args, **kwargs) -> None:
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)

    def __iter__(self) -> Iterator[Any]:
        for name in self.__slots__:
            yield getattr(self, name)

    def __repr__(self):
        values = ', '.join(
            f'{name}={value!r}'
            for name, value in zip(self.__slots__, self)
        )

        cls_name = self.__class__.__name__
        return f'{cls_name}({values})'


    cls_attrs = dict(
        __slots__=slots,
        __init__=__init__,
        __iter__=__iter__,
        __repr__=__repr__,
    )

    return type(cls_name, (), cls_attrs)

In [6]:
Dog = record_factory('Dog', 'name weight owner')
rex = Dog('Rex', 30, 'Bob')
rex

Dog(name='Rex', weight=30, owner='Bob')

## Metaclass

In [1]:
class Meta(type):
    def __new__(cls, name: str, bases:tuple[type], dct: dict):
        print(f"Creating class {name}")
        return super().__new__(cls, name, bases, dct)


class MyClass(metaclass=Meta):
    pass

Creating class MyClass


In [4]:
from collections import OrderedDict

registry = {}

class Meta(type):
    @classmethod
    def __prepare__(mcls, name, bases):
        return OrderedDict()

    def __new__(mcls, name, bases, namespace):
        """
        - register classes
        - add / modify class attributes
        """
        print(f"Defining class {name}")
        namespace['injected'] = lambda self: "Injected"
        cls = super().__new__(mcls, name, bases, namespace)
        return cls

    def __init__(cls, name, bases, namespace):
        """
        - validation of the final class object
        - register classes
        """
        print(f"Initializing class {name}")
        registry[name] = cls
        super().__init__(name, bases, namespace)

    def __call__(cls, *args, **kw):
        """
        - enforce singleton pattern
        - instance caching
        """
        print(f"Creating instance of {cls.__name__}")
        return super().__call__(*args, **kw)

    def __setattr__(cls, key, value):
        """
        - restrict adding attributes
        - track changes to class-level fields
        """
        if key.startswith("_x"):
            raise AttributeError(f"'{key}' is forbidden")
        super().__setattr__(key, value)


class A(metaclass=Meta):
    x = 5


Defining class A
Initializing class A


In [5]:
a = A()

Creating instance of A
